<h1><center>  Hourly Energy Consumption PJM USA </center></h1>

### Importing Libraries

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import random
from tabulate import tabulate
import warnings
warnings.filterwarnings("ignore")
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import TimeSeriesSplit
from math import sqrt
from sklearn.metrics import mean_squared_error,mean_absolute_error
import sklearn.metrics as metrics
from sklearn.ensemble import RandomForestRegressor
from tqdm import tqdm
from datetime import datetime
from prettytable import PrettyTable
import shutil,os
import pickle
import joblib

### Final Pipeline

In [2]:
def final_fun_1(X):
    '''This function will take details about energy comsumption as input and returns a prediction
    The input include : PJME_MW,lag_feature1,lag_feature2,hour_sin_feature,hour_cos_feature,rolling_day_mean'''
    
    #Loading best model
    best_model = joblib.load("random_forest1.joblib")
    
    #Predictions
    y_pred = best_model.predict(X);
    
    #Creating dataframe which contains input and predictions
    column = ['lag_feature1','hour_cos_feature','lag_feature2','hour_sin_feature','rolling_day_mean']
    pjme = pd.DataFrame(X , columns = column)
    pjme['Predicted'] = y_pred
    
    return pjme

In [3]:
def final_fun_2(X,Y):
    '''This function evaperformanve by comparing model prediction with actual values'''
    
    final_prediction = final_fun_1(X)
    y_pred = final_prediction['Predicted']
    y_actual = np.expm1(np.array(Y))
    y_predict = np.expm1(np.array(y_pred))
    mape = np.mean(np.abs((y_actual-y_predict)/y_actual))*100
    mae = mean_absolute_error(y_actual,y_predict)
    mse = mean_squared_error(y_actual,y_predict)
    rmse = sqrt(mean_squared_error(y_actual,y_predict))
    
    print("Root Mean Square Error: ",rmse)
    print("Mean Absolute Error: ",mae)
    print("Mean Absolute Percentage Error: ",mape," %")
    print("Mean Absolute Error: ",mse)

In [4]:
pjme_df = pd.read_pickle('final_pjm_stationary.pkl')
X = pjme_df[['lag_feature1','hour_cos_feature','lag_feature2','hour_sin_feature','rolling_day_mean']]
Y = pjme_df['PJME_MW'].values

In [5]:
X = X[:1].values
Y =Y[:1]

In [6]:
%%time
%%capture

# Examine final_fun_1
result = final_fun_1(X);

Wall time: 41.9 s


In [7]:
result

,lag_feature1,hour_cos_feature,lag_feature2,hour_sin_feature,rolling_day_mean,Predicted
0,0.468011,0.707107,0.486405,0.707107,0.556635,10.25472


In [8]:
# Examine final_fun_2
final_fun_2(X,Y)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done  64 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 280 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 640 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done 874 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done 1144 tasks      | elapsed:    0.3s
[Parallel(n_jobs=4)]: Done 1250 out of 1250 | elapsed:    0.3s finished


Root Mean Square Error:  40.35309505195619
Mean Absolute Error:  40.35309505195619
Mean Absolute Percentage Error:  0.1422135508438985  %
Mean Absolute Error:  1628.3722802722111


### Model Deployment

In [ ]:
import flask
import pandas as pd
from joblib import dump, load
import numpy as np


with open(f'random_forest1.joblib', 'rb') as f:
    model = load(f)


app = flask.Flask(__name__, template_folder='templates')


@app.route('/', methods=['GET', 'POST'])
def main():
    if flask.request.method == 'GET':
        return (flask.render_template('main.html'))

    if flask.request.method == 'POST':
        lag_feature1 = flask.request.form['lag_feature1']
        hour_cos_feature = flask.request.form['hour_cos_feature']
        lag_feature2 = flask.request.form['lag_feature2']
        hour_sin_feature = flask.request.form['hour_sin_feature']
        rolling_day_mean = flask.request.form['rolling_day_mean']
        

        inputs = pd.DataFrame([[lag_feature1, hour_cos_feature, lag_feature2, hour_sin_feature, rolling_day_mean]],
                                       columns=['lag_feature1', 'hour_cos_feature', 'lag_feature2', 'hour_sin_feature', 'rolling_day_mean'],
                                                
                                       dtype='float',
                                       index=['input'])

        predictions = model.predict(inputs)[0]
        predictions = np.round(np.expm1(predictions),2)
        print(predictions)

        return flask.render_template('main.html', original_input={'FirstLagFeature': lag_feature1, 'HourlyCosineFeature': hour_cos_feature, 'SecondLagFeature': lag_feature2, 'HourlySineFeature': hour_sin_feature, 'DayRollingFeature': rolling_day_mean},
                                     result=predictions)


if __name__ == '__main__':
    app.run(host='127.0.0.1', port=5000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [03/Jan/2022 17:26:02] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Jan/2022 17:26:02] "GET /style.css HTTP/1.1" 404 -
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done  64 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 280 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 640 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 874 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 1144 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done 1250 out of 1250 | elapsed:    0.2s finished
127.0.0.1 - - [03/Jan/2022 17:26:34] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [03/Jan/2022 17:26:34] "GET /style.css HTTP/1.1" 404 -


28415.92


[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done  64 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 280 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 640 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 874 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 1144 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done 1250 out of 1250 | elapsed:    0.2s finished
127.0.0.1 - - [03/Jan/2022 17:27:20] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [03/Jan/2022 17:27:20] "GET /style.css HTTP/1.1" 404 -


31292.29


Model Deployment video [click here](https://youtu.be/HDebK1WxRmQ)